In [2]:
from pwn import *
from random import randint
from Crypto.Util.number import *
from hashlib import sha256, md5
from ecdsa import SECP256k1, ellipticcurve
from ecdsa.ecdsa import Public_key, Private_key, Signature

In [26]:
r = remote('edu-ctf.csie.org', 42074)
Px, Py = r.recvline().decode()[5:-2].split(', ')
Px, Py

[x] Opening connection to edu-ctf.csie.org on port 42074
[x] Opening connection to edu-ctf.csie.org on port 42074: Trying 140.112.31.97
[+] Opening connection to edu-ctf.csie.org on port 42074: Done


('11355180168742017688982524058427774693828732603029202752270459130329695603577',
 '63514070290159100640982136941659289533654882097601939291308482440070636236749')

In [27]:
E = SECP256k1
G, n = E.generator, E.order
pubkey = Public_key(G, ellipticcurve.PointJacobi(E, int(Px), int(Py), 1, n))

In [28]:
def send_opt_1(msg):
    r.sendlineafter(b"3) exit", b"1")
    r.sendlineafter(b"Who are you?\n", msg.encode())
    sig = r.recvline()
    sig_r, sig_s = sig.decode()[1:-2].split(', ')
    return int(sig_r), int(sig_s)

m1 = '123'
m2 = '456'
h1 = bytes_to_long(sha256(m1.encode()).digest())
h2 = bytes_to_long(sha256(m2.encode()).digest())
r1, s1 = send_opt_1(m1)
r2, s2 = send_opt_1(m2)

b'(36627077418143416215002081252753048039097746978375100127966087866791169324362, 87532789933868186749570805667085591927291470753494541697474771725139779944055)\n'
b'(4453469421302686120308134350445995419560882000108123272936402662738152502028, 64000197955216668037634786675103484823041512861594482527708367044957296000472)\n'


In [29]:
(r1, s1), (r2, s2)

((36627077418143416215002081252753048039097746978375100127966087866791169324362,
  87532789933868186749570805667085591927291470753494541697474771725139779944055),
 (4453469421302686120308134350445995419560882000108123272936402662738152502028,
  64000197955216668037634786675103484823041512861594482527708367044957296000472))

$s_1 k_1 - h_1 = d * r_1$ and $s_2 k_2 - h_2 = d * r_2$

$(s_1 k_1 - h_1) / (s_2 k_2 - h_2) = r_1/r_2$

$\to (s_1 k_1 - h_1) *r_2 = (s_2 k_2 - h_2) * r_1$

$\to s_1 k_1 r_2 - h_1 r_2 = s_2 k_2 r_1 - h_2 r_1$

$\to s_1 k_1 r_2 - s_2 k_2 r_1 - h_1 r_2 + h_2 r_1 = 0$

$\to k_1 - s_1^{-1} r_2^{-1} s_2 r_1 k_2 - s_1^{-1} h_1 + s_1^{-1} r_2^{-1} h_2 r_1 = 0$

let $t = - s_1^{-1} r_2^{-1} s_2 r_1$ and $u = s_1^{-1} r_2^{-1} h_2 r_1 - s_1^{-1} h_1$

$\to k_1 + t k_2 + u = 0 \ (mod \ n)$

In [30]:
F = Zmod(n)
r1 = F(r1)
r2 = F(r2)
s1 = F(s1)
s2 = F(s2)
h1 = F(h1)
h2 = F(h2)

In [31]:
t = - (s1^(-1) * s2 * r1 * r2^(-1))
u = s1^(-1) * r1 * h2 * r2^(-1)  - s1^(-1) * h1

t, u

(87575850598287155698864998687778268789448367850777566220582845610534308901569,
 86768715067076168970831900382500140925837559481630216440922267711181563281830)

$k_1 + t k_2 + u = 0 \ (mod \ n)$

$\to (a + p_1) + t (a + p_2) + u = 0 \ (mod \ n)$

$\to p_1 + t p_2 + (1 + t) a + u = 0 \ (mod \ n)$

$|p_1|, [p_2| < 128$

In [32]:
a = int(md5(b'secret').hexdigest(), 16) << 128

In [42]:
L = matrix(ZZ, [[n, 0, 0], [t, 1, 0], [a * (1+t) + u, 0, 2^128]])
L.LLL()

[-159821575041606419486715044730765694571   38471066189327077375200861893040212482                                        0]
[  -2857886673820326394879990107096782912  263224668561855954544352848747787348104  340282366920938463463374607431768211456]
[  34690908382911168791836238971999550053  453621219875384669069022410590153941021 -340282366920938463463374607431768211456]

In [34]:
v = L.LLL()[1]
p1 = -v[0]
p2 = v[1]
k1 = a + p1
k2 = a + p2
k1, k2

(42853347383522459682061542032602724326549630868226670186319534033805907368000,
 42853347383522459682061542032602724326809997650114705814469006892446597933192)

In [35]:
(s1 * k1 - h1) * r1^(-1), (s2 * k2 - h2) * r2^(-1)

(107969767335906210582810195697128791765421522667432316637444869637942061281090,
 107969767335906210582810195697128791765421522667432316637444869637942061281090)

In [36]:
d = (s2 * k2 - h2) * r2^(-1)
(int(d)*G).x(), Px

(mpz(11355180168742017688982524058427774693828732603029202752270459130329695603577),
 '11355180168742017688982524058427774693828732603029202752270459130329695603577')

In [38]:
h = sha256('Kuruwa'.encode()).digest()
k = int(md5(b'secret').hexdigest() + md5(long_to_bytes(int(d)) + h).hexdigest(), 16)
prikey = Private_key(pubkey, d)
sig = prikey.sign(bytes_to_long(h), k)
str(sig.r), str(sig.s)

('63825320594311623643769791740580330406314025161475446258801230035581964411195',
 '113387105831969992999296008760571004273107175410665454360839926815706465618861')

In [39]:
r.recvline()
r.recvline()
r.recvline()
r.recvline()

b'3) exit\n'

In [40]:
r.sendline(b"2")
r.sendlineafter(b'username: ', b"Kuruwa")
r.sendlineafter(b'r: ', str(sig.r).encode())
r.sendlineafter(b's: ', str(sig.s).encode())

b's: '

In [41]:
r.recvline()

b'FLAG{adfc9b68bd6ec6dbf6b3c9ddd46aafaea06a97ee}\n'